In [5]:
import pandas as pd

df = pd.read_csv("orders.csv")

df = df[(df['Product'] == 'MIS') & (df['Status'] == 'COMPLETE')].copy()

df['Qty.'] = df['Qty.'].str.split('/').str[0].astype(int)

df['Turnover'] = df['Qty.'] * df['Avg. price']


df['Brokerage'] = df.apply(lambda row: min(row['Turnover'] * 0.0003, 20), axis=1)


df['STT/CTT'] = df.apply(lambda row: row['Turnover'] * 0.00025 if row['Type'] == 'SELL' else 0, axis=1)


df['ETC'] = df['Turnover'] * 0.0000375


df['SEBI'] = df['Turnover'] * 0.0000001


df['GST'] = (df['Brokerage'] + df['ETC']) * 0.18


df['Stamp Charges'] = df.apply(lambda row: min(row['Turnover'] * 0.00003, 300) if row['Type'] == 'BUY' else 0, axis=1)


df['Total Charges'] = df[['Brokerage', 'STT/CTT', 'ETC', 'SEBI', 'GST', 'Stamp Charges']].sum(axis=1)


df_summary = df.groupby(['Instrument', 'Type']).agg({
    'Qty.': 'sum',
    'Avg. price': 'mean',
    'Turnover': 'sum',
    'Brokerage': 'sum',
    'STT/CTT': 'sum',
    'ETC': 'sum',
    'SEBI': 'sum',
    'GST': 'sum',
    'Stamp Charges': 'sum',
    'Total Charges': 'sum'
}).reset_index()


df_overall = df.groupby(['Instrument']).agg({
    'Turnover': 'sum',
    'Total Charges': 'sum'
}).reset_index()
df_overall['Gross PnL'] = df.groupby(['Instrument'])['Turnover'].sum().values
df_overall['Net PnL'] = df_overall['Gross PnL'] - df_overall['Total Charges']
df_overall['% Charges on Gross PnL'] = (df_overall['Total Charges'] / df_overall['Gross PnL']) * 100


with pd.ExcelWriter("trade_summary.xlsx") as writer:
    df.to_excel(writer, sheet_name="Individual Trades", index=False)
    df_summary.to_excel(writer, sheet_name="Stock Type Analysis", index=False)
    df_overall.to_excel(writer, sheet_name="Overall Summary", index=False)

print("Excel summary file generated successfully!")


Excel summary file generated successfully!
